### １０章　ディープラーニングモデル

### 必要ライブラリの導入

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib -qq

In [ ]:
# 共通事前処理

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# Numpyで浮動小数点表示を固定する
np.set_printoptions(formatter={'float': '{:0.3f}'.format})

# データフレーム表示用関数
from IPython.display import display

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.3f}'.format

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

## 10-7 プログラム実装(データ準備・加工)

### データ読み込み

#### 読み込み

In [ ]:
# データ読み込み

# データ読み込み用ライブラリのインポート
from torchvision.datasets import MNIST
from torchvision import transforms

# 訓練データ
train_dataset = MNIST(root="./data", train=True, download=True)

# テストデータ
test_dataset = MNIST(root="./data", train=False, download=True)

# NumPy 配列に変換
x_train_np = train_dataset.data.numpy().reshape(-1, 28*28)
y_train_np = train_dataset.targets.numpy()

x_test_np = test_dataset.data.numpy().reshape(-1, 28*28)
y_test_np = test_dataset.targets.numpy()

#### 内容表示

In [ ]:
# 内容表示

# 4つの変数のshape確認
print(x_train_np.shape, y_train_np.shape, x_test_np.shape, y_test_np.shape)

# y_test_npの先頭10要素
print(y_test_np[:10])

# x_test_npの最初の要素
print(x_test_np[0])

#### イメージ表示

In [ ]:
# イメージ表示
N1 = 10 # 横個数
N2 = 5  # 縦個数
N = N1 * N2 # 10*5 = 50個

# テストデータからランダムに50個のイメージを抽出
# 再現性があるようにseed値をセットしておく
np.random.seed(12)
indexes = np.random.choice(y_test_np.shape[0], N, replace=False)
x_selected = x_test_np[indexes]
y_selected = y_test_np[indexes]

# 50個の描画領域を設定
plt.figure(figsize=(N1, N2), tight_layout=True)
for i in range(N):
    # 個別イメージの描画位置指定
    ax = plt.subplot(N2, N1, i + 1)

    # イメージ表示
    ax.imshow(x_selected[i].reshape(28, 28), cmap='gray_r')

    # 正解ラベルを画像のタイトルとして表示
    ax.set_title(f'{int(y_selected[i]):d}', fontsize=12)

    # 軸ラベル非表示
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### データ加工

#### xの正規化

In [ ]:
# xの正規化
x_train_norm_np = x_train_np / 255.0
x_test_norm_np = x_test_np / 255.0

# 先頭にダミー変数を追加
x_train_dum_np = np.insert(x_train_norm_np, 0, 1, axis=1)
x_test_dum_np = np.insert(x_test_norm_np, 0, 1, axis=1)

# shapeを表示
print(x_train_dum_np.shape, x_test_dum_np.shape)

# 結果の先頭要素を表示
print('x_train_dum_np[0]\n', x_train_dum_np[0])

#### 正解値のOne Hotベクトル化

In [ ]:
# 正解値のOne Hotベクトル化

# OneHotEncoderのインポート
from sklearn.preprocessing import OneHotEncoder

# one hot encoderインスタンスの生成
ohe = OneHotEncoder(sparse_output=False,categories='auto')

# y_train_npのOne Hotベクトル化
y_train_ohe_np = ohe.fit_transform(y_train_np.reshape(-1, 1))

# y_test_npのOne Hotベクトル化
y_test_ohe_np = ohe.transform(y_test_np.reshape(-1, 1))

# 各変数のshape確認
print('y_train One Hot Vector化前', y_train_np.shape)
print('y_train One Hot Vector化後', y_train_ohe_np.shape)
print('y_test One Hot Vector化前', y_test_np.shape)
print('y_test One Hot Vector化後', y_test_ohe_np.shape)

## 10-8 プログラム実装(GPU利用)

#### Tensor型に変換

In [ ]:
# Tensor型に変換

# PyTorchのインポート
import torch

# NumPy変数をTensor変数に変換
x_train = torch.Tensor(x_train_dum_np).float()
y_train = torch.Tensor(y_train_np).float()
y_train_ohe = torch.Tensor(y_train_ohe_np).float()
x_test = torch.Tensor(x_test_dum_np).float()
y_test = torch.Tensor(y_test_np).float()
y_test_ohe = torch.Tensor(y_test_ohe_np).float()

#### GPU 存在チェック

In [ ]:
# GPU存在チェック

# GPUが利用可能かどうかのチェック
device = torch.device("cuda:0" \
if torch.cuda.is_available() else "cpu")

# 利用可能な場合は"cuda:0"が出力される
print(device)

#### Tensor型データをGPUに転送

In [ ]:
# Tensor型データをGPUに転送

# 訓練データをGPUに転送
x_train_gpu = x_train.to(device)
y_train_gpu = y_train.to(device)
y_train_ohe_gpu = y_train_ohe.to(device)

# テストデータをGPUに転送
x_test_gpu = x_test.to(device)
y_test_gpu = y_test.to(device)
y_test_ohe_gpu = y_test_ohe.to(device)

## 10-9 プログラム実装(ミニバッチ学習法・関数定義)

### ミニバッチ学習法

#### データローダーの定義

In [ ]:
# データローダーの定義

# 必要ライブラリのインポート
from torch.utils.data import DataLoader, TensorDataset

# 0から19までの1次元配列をTensorDatasetとして定義
dataset = TensorDataset(torch.arange(20))

# datasetを5個ずつのグループで取り出すようにデータローダーを定義
loader = DataLoader(dataset, batch_size=5, shuffle=True)

#### データローダーの利用イメージ

In [ ]:
# データローダーの利用イメージ
# 繰り返し回数(nb_epoch)=2で上記ローダを利用する

# 繰り返し回数の定義
nb_epoch = 2
for epoch in range(nb_epoch):
    print(f'epoch = {epoch}')

    # データローダーから5個ずつのグループにしたデータをbatchとして取得
    for batch in loader:
        print(batch[0].numpy())

### 関数定義

#### 汎用関数
シグモイド関数とsoftmax関数はPyTorch標準関数を利用

In [ ]:
# 汎用関数

# PyTorch標準関数用ライブラリのインポート
import torch.nn as nn

# シグモイド関数の定義
sigmoid = nn.Sigmoid()

# softmax関数の定義
softmax = nn.Softmax(dim=1)

#### 交差エントロピー関数

In [ ]:
# 交差エントロピー関数
# PyTorchの交差エントロピー関数はわかりにくいのでスクラッチで実装

def cross_entropy(yt_ohe, yp_ohe):
    return -torch.mean(torch.sum(yt_ohe * torch.log(yp_ohe), dim=1))

#### 評価関数
損失と精度を計算

In [ ]:
# 評価関数
# 重み行列V, Wから学習中の損失と精度を計算

# accuracy_scoreは精度評価のための関数
from sklearn.metrics import accuracy_score

def evaluate(x_test, y_test, y_test_ohe, x_dum, V, W):

    # テストデータに対して隠れ層の値を計算
    b_test = sigmoid(x_test @ V)

    # ダミー変数追加
    b1_test = torch.cat((x_dum, b_test), dim=1)

    # 予測値の計算(確率値)
    yp_test_ohe = softmax(b1_test @ W)

    # 予測クラス計算(0から9の整数)
    yp_test = torch.argmax(yp_test_ohe, dim=1)

    # 損失計算(item関数でスカラー化)
    loss = cross_entropy(y_test_ohe, yp_test_ohe).item()

    # 精度計算(item関数でスカラー化)
    score = (y_test == yp_test).float().mean().item()

    return score, loss

## 10-10 プログラム実装(学習その1)

### 学習

#### 学習関数1

In [ ]:
# 学習関数1

def train_mlp(
    hidden_units=100, epochs=500, batch_size=500, alpha_np=0.01,
    his_unit=10):
    """
    2層（入力→Sigmoid→出力Softmax）の単純なMLPを、
    勾配降下法で学習する関数。
    返り値: (V, W, history[epoch, loss, acc])
    """
    # M: 訓練用系列データ総数
    # D: 入力データ次元数
    M, D  = x_train.shape
    # N: 分類クラス数
    N = y_train_ohe.shape[1]
    # 隠れ層のノード数
    H = hidden_units
    H1 = H + 1
    # 重み行列の初期設定
    np.random.seed(123)
    V_np = np.ones((D, H))
    W_np = np.ones((H1, N))
    # GPU転送
    alpha = torch.tensor(alpha_np).float().to(device)
    V = torch.tensor(V_np).float().to(device)
    W = torch.tensor(W_np).float().to(device)

    # 学習時のダミー変数
    x_dum_f = torch.ones((batch_size,1)).float().to(device)
    # 予測時のダミー変数
    x_dum_p = torch.ones((len(x_test),1)).float().to(device)

    # 評価結果記録用 (損失関数値と精度)
    history = np.zeros((0, 3))

    # データローダー初期化(ミニバッチ処理用)
    dataset = TensorDataset(torch.arange(M))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # 繰り返し回数カウンタ初期化
    epoch = 0

    # 繰り返し計算
    while epoch < epochs:
        # 学習対象の選択(ミニバッチ学習法)
        for batch in loader:
            index = batch[0]
            x = x_train_gpu[index]
            yt = y_train_ohe_gpu[index]

            # 予測計算 (順伝播)
            a = x @ V
            b = sigmoid(a)
            b1 = torch.cat((x_dum_f, b), dim=1)
            u = b1 @ W
            yp = softmax(u)

            # 誤差計算 (逆伝播)
            yd = yp - yt
            bd = b * (1-b) * (yd @ W[1:].T)

            # 勾配計算
            grad_W = (b1.T @ yd) / batch_size
            grad_V = (x.T  @ bd) / batch_size

            # パラメータ更新
            W -= alpha * grad_W
            V -= alpha * grad_V

        score, loss = evaluate(x_test_gpu, y_test_gpu, y_test_ohe_gpu, x_dum_p, V, W)
        history = np.vstack((history, np.array([epoch, loss, score])))
        epoch = epoch + 1
        if (epoch-1) % his_unit == 0:
            print(f'epoch = {epoch-1} loss = {loss:.04f} score = {score:.04f}')

    return W, V, history

#### 学習

In [ ]:
%%time
# 学習

# 学習パラメータ
hidden_units = 100  # 隠れ層サイズ
alpha_np = 0.01     # 学習率
batch_size = 500    # バッチサイズ
epochs = 500        # 繰り返し数
his_unit = 10       # 画面表示頻度

# 繰り返し処理
W, V, history1 = train_mlp(
    hidden_units=hidden_units,
    alpha_np=alpha_np, batch_size=batch_size,
    epochs=epochs, his_unit=his_unit)


### 結果分析

#### 損失と精度確認

In [ ]:
# 損失と精度の確認
# 損失と精度の初期値と最終値を確認
print(f'初期状態: 関数:{history1[0,1]:.04f} \
精度:{history1[0,2]:.04f}')
print(f'最終状態: 関数:{history1[-1,1]:.04f} \
精度:{history1[-1,2]:.04f}')

#### 学習曲線(損失)

In [ ]:
# 学習曲線(損失)

plt.figure(figsize=(6,6))
plt.plot(history1[:,0], history1[:,1], 'b')
plt.ylim(0,2.5)
plt.xlabel('繰り返し回数')
plt.ylabel('損失')
plt.title('学習曲線(損失)')
plt.grid()
plt.show()

#### 学習曲線(精度)

In [ ]:
# 学習曲線(精度)

plt.figure(figsize=(6,6))
plt.plot(history1[:,0], history1[:,2], 'b')
plt.ylim(0,1)
plt.xlabel('繰り返し回数')
plt.ylabel('精度')
plt.title('学習曲線(精度)')
plt.grid()
plt.show()

## 10-11 プログラム実装(学習その2)

### 学習( パラメータ初期値変更)

#### 重み行列初期化改訂版

In [ ]:
# 重み行列初期化改訂版
hidden_units = 100  # 隠れ層サイズ
M, D  = x_train.shape
N = y_train_ohe.shape[1]
H = hidden_units
H1 = H + 1
np.random.seed(123)
V_np = np.random.randn(D, H) / np.sqrt(D / 2)
W_np = np.random.randn(H1, N) / np.sqrt(H1 / 2)

# 内容の一部確認
print(V_np[:2,:5])
print(W_np[:2,:5])

#### 学習関数2

In [ ]:
# 学習関数2

def train_mlp2(
    hidden_units=100, epochs=500, batch_size=500, alpha_np=0.01,
    his_unit=10):
    """
    2層（入力→Sigmoid→出力Softmax）の単純なMLPを、
    勾配降下法で学習する関数。
    返り値: (V, W, history[epoch, loss, acc])
    """
    # M: 訓練用系列データ総数
    # D: 入力データ次元数
    M, D  = x_train.shape
    # N: 分類クラス数
    N = y_train_ohe.shape[1]
    # 隠れ層のノード数
    H = hidden_units
    H1 = H + 1
    # 重み行列初期化改訂版
    np.random.seed(123)
    V_np = np.random.randn(D, H) / np.sqrt(D / 2)
    W_np = np.random.randn(H1, N) / np.sqrt(H1 / 2)
    # GPU転送
    alpha = torch.tensor(alpha_np).float().to(device)
    V = torch.tensor(V_np).float().to(device)
    W = torch.tensor(W_np).float().to(device)

    # 学習時のダミー変数
    x_dum_f = torch.ones((batch_size,1)).float().to(device)
    # 予測時のダミー変数
    x_dum_p = torch.ones((len(x_test),1)).float().to(device)

    # 評価結果記録用 (損失関数値と精度)
    history = np.zeros((0, 3))

    # データローダー初期化(ミニバッチ処理用)
    dataset = TensorDataset(torch.arange(M))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # 繰り返し回数カウンタ初期化
    epoch = 0

    # 繰り返し計算
    while epoch < epochs:
        # 学習対象の選択(ミニバッチ学習法)
        for batch in loader:
            index = batch[0]
            x = x_train_gpu[index]
            yt = y_train_ohe_gpu[index]

            # 予測計算 (順伝播)
            a = x @ V
            b = sigmoid(a)
            b1 = torch.cat((x_dum_f, b), dim=1)
            u = b1 @ W
            yp = softmax(u)

            # 誤差計算 (逆伝播)
            yd = yp - yt
            bd = b * (1-b) * (yd @ W[1:].T)

            # 勾配計算
            grad_W = (b1.T @ yd) / batch_size
            grad_V = (x.T  @ bd) / batch_size

            # パラメータ更新
            W -= alpha * grad_W
            V -= alpha * grad_V

        score, loss = evaluate(x_test_gpu, y_test_gpu, y_test_ohe_gpu, x_dum_p, V, W)
        history = np.vstack((history, np.array([epoch, loss, score])))
        epoch = epoch + 1
        if (epoch-1) % his_unit == 0:
            print(f'epoch = {epoch-1} loss = {loss:.04f} score = {score:.04f}')

    return W, V, history

#### 学習

In [ ]:
%%time
# 学習

# 学習パラメータ
hidden_units = 100  # 隠れ層サイズ
alpha_np = 0.01     # 学習率
batch_size = 500    # バッチサイズ
epochs = 500        # 繰り返し数
his_unit = 10       # 画面表示頻度

# 繰り返し処理
W, V, history2 = train_mlp2(
    hidden_units=hidden_units,
    alpha_np=alpha_np, batch_size=batch_size,
    epochs=epochs, his_unit=his_unit)

### 結果分析

#### 損失と精度の確認

In [ ]:
# 損失と精度の確認
# 損失と精度の初期値と最終値を確認
print(f'初期状態: 損失:{history2[0,1]:.04f} \
精度:{history2[0,2]:.04f}')
print(f'最終状態: 損失:{history2[-1,1]:.04f} \
精度:{history2[-1,2]:.04f}')

#### 学習曲線(損失)

In [ ]:
# 学習曲線(損失)

plt.figure(figsize=(6,6))
plt.plot(history2[:,0], history2[:,1], 'b')
plt.xlabel('繰り返し回数')
plt.ylabel('損失')
plt.title('学習曲線(損失)')
plt.grid()
plt.show()

#### 学習曲線(精度)

In [ ]:
# 学習曲線(精度)

plt.figure(figsize=(6,6))
plt.plot(history2[:,0], history2[:,2], 'b')
plt.ylim(0.4,1)
plt.xlabel('繰り返し回数')
plt.ylabel('精度')
plt.title('学習曲線(精度)')
plt.grid()
plt.show()

#### イメージで確認

In [ ]:
# イメージで確認

N1 = 10 # 横個数
N2 = 5  # 縦個数
N = N1 * N2 # 10*5 = 50個

# テストデータからランダムに50個のイメージを抽出
# 再現性があるようにseed値をセットしておく
np.random.seed(12)
indexes = np.random.choice(y_test.shape[0], N, replace=False)
x_selected = x_test_gpu[indexes]
x_selected_show = x_test_np[indexes]
y_selected = y_test_gpu[indexes]

# ダミー変数の準備
x_dum3 = torch.ones((N,1)).float().to(device)

# 予測値の計算
# テストデータに対して隠れ層の値を計算
b_test = sigmoid(x_selected @ V)

# ダミー変数追加
b1_test = torch.cat((x_dum3, b_test), dim=1)

# 予測値の計算(確率値)
yp_test_ohe = softmax(b1_test @ W)

# 予測クラス計算(0から9の整数)
yp_test = torch.argmax(yp_test_ohe, dim=1)

# 50個の描画領域を設定
plt.figure(figsize=(N1, N2), tight_layout=True)
for i in range(N):

    # 描画位置指定
    ax = plt.subplot(N2, N1, i + 1)

    # 画像イメージ表示
    ax.imshow(x_selected_show[i].reshape(28, 28),cmap='gray_r')

    # 正解ラベルと予測値
    true_label = int(y_selected[i])
    pred_label = int(yp_test[i])

    # タイトル色を条件で変更
    title_color = 'blue' if true_label != pred_label else 'black'

    # タイトル表示
    ax.set_title(f'{true_label}:{pred_label}', fontsize=14, color=title_color)

    # 軸ラベル非表示
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## 10-12 プログラム実装(学習その3)

### 学習

#### 学習

In [ ]:
%%time
# 学習

# 学習パラメータ
hidden_units = 100  # 隠れ層サイズ
alpha_np = 0.01     # 学習率
#batch_size = 500   # バッチサイズ前回値
batch_size = 100    # バッチサイズ今回値
epochs = 500        # 繰り返し数
his_unit = 10       # 画面表示頻度

# 繰り返し処理
W, V, history3 = train_mlp2(
    hidden_units=hidden_units,
    alpha_np=alpha_np, batch_size=batch_size,
    epochs=epochs, his_unit=his_unit)


### 結果分析

#### 損失と精度の確認

In [ ]:
# 損失と精度の確認
# 損失と精度の初期値と最終値を確認
print(f'初期状態: 損失:{history3[0,1]:.04f} \
精度:{history3[0,2]:.04f}')
print(f'最終状態: 損失:{history3[-1,1]:.04f} \
精度:{history3[-1,2]:.04f}')

#### 学習曲線(損失)

In [ ]:
# 学習曲線(損失)

plt.figure(figsize=(6,6))
plt.plot(history3[:,0], history3[:,1], 'b')
plt.xlabel('繰り返し回数')
plt.ylabel('損失')
plt.title('学習曲線(損失)')
plt.grid()
plt.show()

#### 学習曲線(精度)

In [ ]:
# 学習曲線(精度)

plt.figure(figsize=(6,6))
plt.plot(history3[:,0], history3[:,2], 'b')
plt.ylim(0.4,1)
plt.xlabel('繰り返し回数')
plt.ylabel('精度')
plt.title('学習曲線(精度)')
plt.grid()
plt.show()

#### イメージで確認

In [ ]:
# イメージで確認

N1 = 10 # 横個数
N2 = 5  # 縦個数
N = N1 * N2 # 10*5 = 50個

# テストデータからランダムに50個のイメージを抽出
# 再現性があるようにseed値をセットしておく
np.random.seed(12)
indexes = np.random.choice(y_test.shape[0], N, replace=False)
x_selected = x_test_gpu[indexes]
x_selected_show = x_test_np[indexes]
y_selected = y_test_gpu[indexes]

# ダミー変数の準備
x_dum3 = torch.ones((N,1)).float().to(device)

# 予測値の計算
# テストデータに対して隠れ層の値を計算
b_test = sigmoid(x_selected @ V)

# ダミー変数追加
b1_test = torch.cat((x_dum3, b_test), dim=1)

# 予測値の計算(確率値)
yp_test_ohe = softmax(b1_test @ W)

# 予測クラス計算(0から9の整数)
yp_test = torch.argmax(yp_test_ohe, dim=1)

# 50個の描画領域を設定
plt.figure(figsize=(N1, N2), tight_layout=True)
for i in range(N):

    # 描画位置指定
    ax = plt.subplot(N2, N1, i + 1)

    # 画像イメージ表示
    ax.imshow(x_selected_show[i].reshape(28, 28),cmap='gray_r')

    # 正解ラベルと予測値
    true_label = int(y_selected[i])
    pred_label = int(yp_test[i])

    # タイトル色を条件で変更
    title_color = 'blue' if true_label != pred_label else 'black'

    # タイトル表示
    ax.set_title(f'{true_label}:{pred_label}', fontsize=14, color=title_color)

    # 軸ラベル非表示
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## 10-13 プログラム実装(学習その4)

### 関数定義

#### evaluate2関数
隠れ層2層向けの評価関数

In [ ]:
# evaluate2関数
def evaluate2(x_test, y_test, y_test_ohe, x_dum_p, U, V, W):

    # テストデータに対して隠れ層の値を計算
    b_test = sigmoid(x_test @ U)

    # ダミー変数追加
    b1_test = torch.cat((x_dum_p, b_test), dim=1)

    d_test = sigmoid(b1_test @ V)

    # ダミー変数追加
    d1_test = torch.cat((x_dum_p, d_test), dim=1)

    # 予測値算出
    yp_test_ohe = softmax(d1_test @ W)

    # ラベル値算出
    yp_test = torch.argmax(yp_test_ohe, dim=1)

    # 損失計算(item関数でスカラー化)
    loss = cross_entropy(y_test_ohe, yp_test_ohe).item()

    # 精度計算(item関数でスカラー化)
    score = (y_test == yp_test).float().mean().item()

    return score, loss

### 学習(隠れ層2層化)

#### 学習関数3

In [ ]:
def train_mlp3(
    hidden_units=100, epochs=500, batch_size=500, alpha_np=0.01,
    his_unit=10):
    """
    3層（入力→Sigmoid→出力Softmax）の単純なMLPを、
    勾配降下法で学習する関数。
    返り値: (U, V, W, history[epoch, loss, acc])
    """
    # M: 訓練用系列データ総数
    # D: 入力データ次元数
    M, D  = x_train.shape
    # N: 分類クラス数
    N = y_train_ohe.shape[1]
    # 隠れ層のノード数
    H = hidden_units
    H1 = H + 1
    # 重み行列初期化改訂版
    np.random.seed(123)
    U_np = np.random.randn(D, H) / np.sqrt(D / 2)
    V_np = np.random.randn(H1, H) / np.sqrt(H1 / 2)
    W_np = np.random.randn(H1, N) / np.sqrt(H1 / 2)
    # GPU転送
    alpha = torch.tensor(alpha_np).float().to(device)
    U = torch.tensor(U_np).float().to(device)
    V = torch.tensor(V_np).float().to(device)
    W = torch.tensor(W_np).float().to(device)

    # 学習時のダミー変数
    x_dum_f = torch.ones((batch_size,1)).float().to(device)
    # 予測時のダミー変数
    x_dum_p = torch.ones((len(x_test),1)).float().to(device)

    # 評価結果記録用 (損失関数値と精度)
    history = np.zeros((0, 3))

    # データローダー初期化(ミニバッチ処理用)
    dataset = TensorDataset(torch.arange(M))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # 繰り返し回数カウンタ初期化
    epoch = 0

    # 繰り返し計算
    while epoch < epochs:
        # 学習対象の選択(ミニバッチ学習法)
        for batch in loader:
            index = batch[0]
            x = x_train_gpu[index]
            yt = y_train_ohe_gpu[index]

            # 予測計算 (順伝播)
            a = x @ U
            b = sigmoid(a)
            b1 = torch.cat((x_dum_f, b), dim=1)
            c = b1 @ V
            d = sigmoid(c)
            d1 = torch.cat((x_dum_f, d), dim=1)
            u = d1 @ W
            yp = softmax(u)

            # 誤差計算 (逆伝播)
            yd = yp - yt
            dd = d * (1-d) * torch.tensor(yd @ W[1:].T)
            bd = b * (1-b) * torch.tensor(dd @ V[1:].T)

            # 勾配計算
            grad_W = (d1.T @ yd) / batch_size
            grad_V = (b1.T  @ dd) / batch_size
            grad_U = (x.T @ bd) / batch_size

            # パラメータ更新
            W -= alpha * grad_W
            V -= alpha * grad_V
            U -= alpha * grad_U

        score, loss = evaluate2(x_test_gpu, y_test_gpu, y_test_ohe_gpu,
                                x_dum_p, U, V, W)
        history = np.vstack((history, np.array([epoch, loss, score])))
        epoch = epoch + 1
        if (epoch-1) % his_unit == 0:
            print(f'epoch = {epoch-1} loss = {loss:.04f} score = {score:.04f}')
    return U, W, V, history

#### 学習

In [ ]:
%%time
# 学習

# 学習パラメータ
hidden_units = 100  # 隠れ層サイズ
alpha_np = 0.01     # 学習率
#batch_size = 500   # バッチサイズ前回値
batch_size = 100    # バッチサイズ今回値
epochs = 1000        # 繰り返し数
his_unit = 10       # 画面表示頻度

# 繰り返し処理
U, W, V, history4 = train_mlp3(
    hidden_units=hidden_units,
    alpha_np=alpha_np, batch_size=batch_size,
    epochs=epochs, his_unit=his_unit)


### 結果分析その4

#### 損失と精度の確認

In [ ]:
# 損失と精度の確認
# 損失と精度の初期値と最終値を確認
print(f'初期状態: 損失:{history4[0,1]:.04f} \
精度:{history4[0,2]:.04f}')
print(f'最終状態: 損失:{history4[-1,1]:.04f} \
精度:{history4[-1,2]:.04f}')

#### 学習曲線(損失)

In [ ]:
# 学習曲線(損失)

plt.figure(figsize=(6,6))
plt.plot(history4[:,0], history4[:,1], 'b')
plt.ylim(0,0.5)
plt.xlabel('繰り返し回数')
plt.ylabel('損失')
plt.title('学習曲線(損失)')
plt.grid()
plt.show()

#### 学習曲線(精度)

In [ ]:
# 学習曲線(精度)

plt.figure(figsize=(6,6))
plt.plot(history4[:,0], history4[:,2], 'b')
plt.ylim(0.4,1)
plt.xlabel('繰り返し回数')
plt.ylabel('精度')
plt.title('学習曲線(精度)')
plt.grid()
plt.show()

#### イメージで確認

In [ ]:
# イメージで確認

N1 = 10 # 横個数
N2 = 5  # 縦個数
N = N1 * N2 # 10*5 = 50個

# テストデータからランダムに50個のイメージを抽出
# 再現性があるようにseed値をセットしておく
np.random.seed(12)
indexes = np.random.choice(y_test.shape[0], N, replace=False)
x_selected = x_test_gpu[indexes]
x_selected_show = x_test_np[indexes]
y_selected = y_test_gpu[indexes]

# ダミー変数の準備
x_dum_i = torch.ones((N,1)).float().to(device)

# 予測値の計算(隠れ層1層目)
b_test = sigmoid(x_selected @ U)

# ダミー変数追加
b1_test = torch.cat((x_dum_i, b_test), dim=1)

# 予測値の計算(隠れ層2層目)
d_test = sigmoid(b1_test @ V)

# ダミー変数追加
d1_test = torch.cat((x_dum_i, d_test), dim=1)

# 予測値の計算(確率値)
yp_test_ohe = softmax(d1_test @ W)

# 予測クラス計算(0から9の整数)
yp_test = torch.argmax(yp_test_ohe, dim=1)

# 50個の描画領域を設定
plt.figure(figsize=(N1, N2), tight_layout=True)
for i in range(N):

    # 描画位置指定
    ax = plt.subplot(N2, N1, i + 1)

    # 画像イメージ表示
    ax.imshow(x_selected_show[i].reshape(28, 28),cmap='gray_r')

    # 正解ラベルと予測値
    true_label = int(y_selected[i])
    pred_label = int(yp_test[i])

    # タイトル色を条件で変更
    title_color = 'blue' if true_label != pred_label else 'black'

    # タイトル表示
    ax.set_title(f'{true_label}:{pred_label}', fontsize=14, color=title_color)

    # 軸ラベル非表示
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### バージョン確認

In [ ]:
!pip install watermark -qq
%load_ext watermark
%watermark --iversions